In [37]:
## Libraries
import pandas as pd
import numpy as np
import sklearn as sk
import datetime as dt
import pickle
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import seaborn as sns
import mlflow
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import argparse
import os
from sklearn.ensemble import RandomForestRegressor
import argparse
import pyarrow
from hyperopt import hp, space_eval
from hyperopt.pyll import scope
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from datetime import datetime



In [4]:
## Import Trip Data Yellow Records January, February, March
db = pd.read_parquet("yellow_tripdata_2022-01.parquet")
db2 = pd.read_parquet("yellow_tripdata_2022-02.parquet")
db3 = pd.read_parquet("yellow_tripdata_2022-03.parquet")

In [8]:
## January data
db['PULocationID']=db['PULocationID'].replace(np.nan, -1)
db['DOLocationID']=db['DOLocationID'].replace(np.nan, -1)
categorical = ['PULocationID', 'DOLocationID']
db[categorical] = db[categorical].astype(str)
db ["duration"] = (db.tpep_dropoff_datetime - db.tpep_pickup_datetime) / pd.Timedelta('1 minute')
db = db[db['duration'] <= 60]

In [9]:
## Clean january data and create some variables 
def preprocess(db: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    db['PU_DO'] = db['PULocationID'] + '_' + db['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = db[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [10]:
## February data
db2['PULocationID']=db2['PULocationID'].replace(np.nan, -1)
db2['DOLocationID']=db2['DOLocationID'].replace(np.nan, -1)
categorical = ['PULocationID', 'DOLocationID']
db2[categorical] = db2[categorical].astype(str)
db2 ["duration"] = (db2.tpep_dropoff_datetime - db2.tpep_pickup_datetime) / pd.Timedelta('1 minute')
db2 = db2[db2['duration'] <= 60]

In [11]:
## Clean february
#  data and create some variables 
def preprocess(db2: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    db2['PU_DO'] = db2['PULocationID'] + '_' + db2['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = db2[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [12]:
## March data
db3['PULocationID']=db3['PULocationID'].replace(np.nan, -1)
db3['DOLocationID']=db3['DOLocationID'].replace(np.nan, -1)
categorical = ['PULocationID', 'DOLocationID']
db3[categorical] = db3[categorical].astype(str)
db3 ["duration"] = (db3.tpep_dropoff_datetime - db3.tpep_pickup_datetime) / pd.Timedelta('1 minute')
db3 = db3[db3['duration'] <= 60]

In [13]:
## Clean march data and create some variables 
def preprocess(db3: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    db3['PU_DO'] = db3['PULocationID'] + '_' + db3['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = db3[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [17]:
## Create dependent variable (duration)
target = 'duration'
y_train = db[target].values
y_valid = db2[target].values
y_test = db3[target].values

## Vectorize the data sets    
dv = DictVectorizer()
X_train, dv = preprocess(db, dv, fit_dv=True)
X_valid, _ = preprocess(db2, dv, fit_dv=False)
X_test, _ = preprocess(db3, dv, fit_dv=False)

## Define dump_pickle
def dump_pickle(obj, filename):
    with open(filename, "wb") as f_out:
        return pickle.dump(obj, f_out)

    ## Save dictvectorizer and datasets
    
dump_pickle(dv, os.path.join('/Users/agvg/Documents/DS/MLOPS/MLOPS/Final_Project', "dv.pkl"))
dump_pickle((X_train, y_train), os.path.join('/Users/agvg/Documents/DS/MLOPS/MLOPS/Final_Project', "train.pkl"))
dump_pickle((X_valid, y_valid), os.path.join('/Users/agvg/Documents/DS/MLOPS/MLOPS/Final_Project', "valid.pkl"))
dump_pickle((X_test, y_test), os.path.join('/Users/agvg/Documents/DS/MLOPS/MLOPS/Final_Project', "test.pkl"))

## Load_Pickle 
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

/var/folders/cc/95xr6jg14jx5tk_0cr2yyj900000gn/T/ipykernel_2818/1734027145.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db3['PU_DO'] = db3['PULocationID'] + '_' + db3['DOLocationID']


In [27]:
## MLFLOW
import mlflow


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
mlflow.set_tracking_uri = "sqlite:///mlflow.db"
mlflow.set_experiment = "LR-experiment"

In [25]:
## Run model

HPO_EXPERIMENT_NAME = "LR-Model"
EXPERIMENT_NAME = "LR-Best-Model"

mlflow.autolog()

with mlflow.start_run():
                mlflow.set_tag("model","alexander")
                lr = LinearRegression()
                lr.fit(X_train, y_train)
                y_pred = lr.predict(X_train)
                rmse = mean_squared_error(y_train, y_pred, squared=False)
                mlflow.log_metric("rmse",rmse)

                valid_rmse = mean_squared_error(y_valid, lr.predict(X_valid), squared=False)
                mlflow.log_metric("valid_rmse", valid_rmse)
                print(valid_rmse)
                test_rmse = mean_squared_error(y_test, lr.predict(X_test), squared=False)
                mlflow.log_metric("test_rmse", test_rmse)
                print(test_rmse)

client = MlflowClient()





2022/08/08 10:27:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


5.69904796709038
38.373549303242505


In [40]:
## Model and Run_id
run_id = 'bd6492e7973448fab5e8474c68259c62'
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc_yellow_taxi_regressor")
print(model_uri)

model_name="nyc_yellow_taxi_regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.current_stage}, stage: {version.current_stage}")

## Put the model in staging

model_version = 7
new_stage= "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}")

## Save the models remotely 
data_path='/Users/agvg/Documents/DS/MLOPS/MLOPS/FINAL_PROJECT'
params = '/Users/agvg/Documents/DS/MLOPS/MLOPS/FINAL_PROJECT/artifacts'
preprocessor= '/Users/agvg/Documents/DS/MLOPS/MLOPS/FINAL_PROJECT/models'

client.download_artifacts(run_id=run_id, path=preprocessor, dst_path='.')

## Production 

client.transition_model_version_stage(
    name=model_name,
    version=7,
    stage="Production",
    archive_existing_versions=True
)




Registered model 'nyc_yellow_taxi_regressor' already exists. Creating a new version of this model...
2022/08/08 18:41:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc_yellow_taxi_regressor, version 7
Created version '7' of model 'nyc_yellow_taxi_regressor'.


runs:/bd6492e7973448fab5e8474c68259c62/model
version: None, stage: None
version: Staging, stage: Staging


<ModelVersion: creation_timestamp=1660005698704, current_stage='Production', description='The model version 7 was transitioned to Staging on 2022-08-08', last_updated_timestamp=1660005698806, name='nyc_yellow_taxi_regressor', run_id='bd6492e7973448fab5e8474c68259c62', run_link=None, source='./mlruns/1/bd6492e7973448fab5e8474c68259c62/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>